# RUTINAS DE CONVERSION RASTER TO EXCEL PARA PERU

In [ ]:
pip install rioxarray

In [ ]:
import datetime
import geopandas as gpd
import numpy as np
import os
import pandas as pd
import rioxarray
import xarray as xr
from pyhdf.SD import SD, SDC
from shapely.geometry import mapping


# 1.PRECIPITATION

## 2.1.ACUMULADO MENSUAL

In [1]:


def generate_precip_monthly(nc_file, shapefile_path, start_date, end_date):
    """
    Genera un DataFrame pivot con promedio mensual de precipitación por polígono del shapefile.
    """

    # 1️⃣ Leer shapefile y asegurar CRS WGS84
    gdf = gpd.read_file(shapefile_path).to_crs("EPSG:4326")

    # 2️⃣ Abrir dataset
    ds = xr.open_dataset(nc_file)

    # Variable precipitación
    da = ds['precip']

    # Asegurar que tenga CRS
    da = da.rio.write_crs("EPSG:4326")

    # Ajustar dims a lon/lat (si no reconoce bien)
    da.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)

    # Filtrar por rango de fechas
    da = da.sel(time=slice(start_date, end_date))

    results_list = []

    # 3️⃣ Iterar por cada polígono (ej. provincias)
    for idx, row in gdf.iterrows():
        nombre = row["NOMBPROV"] if "NOMBPROV" in gdf.columns else row["NOMBDEP"]  # Ajusta según shapefile
        geom = [mapping(row["geometry"])]

        try:
            clipped = da.rio.clip(geom, gdf.crs, drop=True, all_touched=True)

            # Sacar promedio por mes (ya que time es mensual en tu nc)
            mean_series = clipped.mean(dim=["latitude", "longitude"]).to_pandas()

            # Guardar resultados
            for date, val in mean_series.items():
                results_list.append({
                    "nombre": nombre,
                    "date": date,
                    "precip_mean": float(val) if val == val else pd.NA  # maneja NaN
                })

        except Exception as e:
            print(f"Error en {nombre}: {e}")

    ds.close()

    # 4️⃣ Convertir resultados a DataFrame
    df = pd.DataFrame(results_list)

    # Columna estilo precip_YYYY_MM
    df["col_name"] = df["date"].dt.strftime("precip_%Y_%m")

    # Pivot table: provincias en filas, fechas en columnas
    df_pivot = df.pivot_table(
        index="nombre",
        columns="col_name",
        values="precip_mean",
        aggfunc="mean",
        fill_value=pd.NA
    ).reset_index()

    return df_pivot

In [ ]:
area = 'DEPARTAMENTOS' # DEPARTAMENTOS, PROVINCIAS Y PROVINCIAS
nc_file = r"D:\Documents_2\LABORAL\2025\INNOVALAB\data\prec\chirps_peru.nc"
shapefile_path = rf"D:\Documents_2\LABORAL\2025\INNOVALAB\shapefile\{area}_inei_geogpsperu_suyopomalia.shp"

start_date = datetime.datetime(2020, 1, 1)
end_date   = datetime.datetime(2024, 12, 31)

df_precip_annual = generate_precip_monthly(nc_file, shapefile_path, start_date, end_date)

# Guardar CSV
df_precip_annual.to_csv("precip_provincias_anual.csv", index=False)
print(df_precip_annual.head())

## # 2.2.ACUMULADO ANUAL

In [2]:


def generate_precip_annual(nc_file, shapefile_path, start_date, end_date):
    """
    Genera un DataFrame pivot con acumulado anual de precipitación por polígono del shapefile.
    """

    # 1️⃣ Leer shapefile y asegurar CRS WGS84
    gdf = gpd.read_file(shapefile_path).to_crs("EPSG:4326")

    # 2️⃣ Abrir dataset
    ds = xr.open_dataset(nc_file)
    da = ds['precip']

    # Asegurar que tenga CRS
    da = da.rio.write_crs("EPSG:4326")
    da.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)

    # Filtrar por rango de fechas
    da = da.sel(time=slice(start_date, end_date))

    results_list = []

    # 3️⃣ Iterar por cada polígono
    for idx, row in gdf.iterrows():
        nombre = row["NOMBPROV"] if "NOMBPROV" in gdf.columns else row["NOMBDEP"]  
        geom = [mapping(row["geometry"])]

        try:
            clipped = da.rio.clip(geom, gdf.crs, drop=True, all_touched=True)

            # Serie temporal: promedio espacial
            mean_series = clipped.mean(dim=["latitude", "longitude"]).to_pandas()

            # Agrupar por año y sacar acumulado
            annual_series = mean_series.groupby(mean_series.index.year).sum()

            # Guardar resultados
            for year, val in annual_series.items():
                results_list.append({
                    "nombre": nombre,
                    "year": year,
                    "precip_annual": float(val) if val == val else pd.NA
                })

        except Exception as e:
            print(f"Error en {nombre}: {e}")

    ds.close()

    # 4️⃣ Convertir resultados a DataFrame
    df = pd.DataFrame(results_list)

    # Columna estilo precip_YYYY
    df["col_name"] = df["year"].astype(str).apply(lambda y: f"precip_{y}")

    # Pivot table: provincias en filas, años en columnas
    df_pivot = df.pivot_table(
        index="nombre",
        columns="col_name",
        values="precip_annual",
        aggfunc="mean",
        fill_value=pd.NA
    ).reset_index()

    return df_pivot

C:\ProgramData\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


col_name         nombre  precip_2023  precip_2024
0               ABANCAY   960.913635   887.732090
1              ACOBAMBA   792.358688   832.228020
2               ACOMAYO   674.086234   761.059544
3                  AIJA   756.451990   471.805914
4         ALTO AMAZONAS  1399.725199  1436.435282


In [ ]:
area = 'DEPARTAMENTOS' # DEPARTAMENTOS, PROVINCIAS Y PROVINCIAS
nc_file = r"D:\Documents_2\LABORAL\2025\INNOVALAB\data\prec\chirps_peru.nc"
shapefile_path = rf"D:\Documents_2\LABORAL\2025\INNOVALAB\shapefile\{area}_inei_geogpsperu_suyopomalia.shp"

start_date = datetime.datetime(2023, 1, 1)
end_date   = datetime.datetime(2024, 12, 31)

df_precip_annual = generate_precip_annual(nc_file, shapefile_path, start_date, end_date)

# Guardar CSV
df_precip_annual.to_csv("precip_distritos_anual_distritos.csv", index=False)
print(df_precip_annual.head())

# 2. LAND SURFACE TEMPERATURE (LST)

In [3]:


def generate_LST_dataframe_by_year(data_dir, shapefile_path, start_date, end_date):
    """
    Genera un DataFrame pivot con promedio de LST MODIS por departamento año por año,
    optimizado para no saturar la memoria.
    """

    def file_to_date(filename):
        base = os.path.basename(filename)
        yday = base.split('.')[1][1:]  # 'A2001335' -> '2001335'
        year = int(yday[:4])
        day_of_year = int(yday[4:])
        return datetime.datetime(year, 1, 1) + datetime.timedelta(day_of_year - 1)

    # Leer shapefile
    gdf = gpd.read_file(shapefile_path).to_crs("EPSG:4326")

    hdf_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".hdf")]
    files_dict = {file_to_date(f): f for f in hdf_files}

    df_final = pd.DataFrame()

    for year in range(start_date.year, end_date.year + 1):
        print(f"Procesando año: {year}")

        year_files = [f for date, f in files_dict.items() if date.year == year and start_date <= date <= end_date]

        if not year_files:
            continue

        results_list = []

        for file in year_files:
            date = file_to_date(file)
            hdf = SD(file, SDC.READ)
            lst_data = hdf.select('LST_Day_CMG')[:]
            lst_c = lst_data * 0.02 - 273.15  # °C

            lat = np.linspace(90, -90, lst_c.shape[0])
            lon = np.linspace(-180, 180, lst_c.shape[1])

            da = xr.DataArray(lst_c, dims=("lat","lon"), coords={"lat": lat, "lon": lon})
            da = da.rename({"lat":"y","lon":"x"}).rio.write_crs("EPSG:4326")

            da = da.where(da > 0)

            for idx, row in gdf.iterrows():
                dep = row["NOMBDEP"]
                geom = [row["geometry"]]
                try:
                    lst_clip = da.rio.clip(geom, gdf.crs, drop=True)
                    mean_val = float(lst_clip.mean().values)
                except Exception:
                    mean_val = np.nan
                results_list.append({"NOMBDEP": dep, "date": date, "LST_mean": mean_val})

        df_year = pd.DataFrame(results_list)
        df_year["col_name"] = df_year["date"].dt.strftime("LST_mean_%Y%_m")
        df_pivot = df_year.pivot_table(index="NOMBDEP", columns="col_name", values="LST_mean").reset_index()

        if df_final.empty:
            df_final = df_pivot
        else:
            df_final = pd.merge(df_final, df_pivot, on="NOMBDEP", how="outer")

        del df_year, df_pivot, da, lst_data, lst_c

    return df_final

In [ ]:
area = 'DEPARTAMENTOS' #DEPARTAMENTOS, PROVINCIAS O DISTRITOS
data_dir = "/content/drive/MyDrive/ACCESO_DIRECTO/LST_MODIS/MODIS_LST"
shapefile_path = f"/content/drive/MyDrive/ACCESO_DIRECTO/SHAPEFILES_PERU/{area}_inei_geogpsperu_suyopomalia.shp"

start_date = datetime.datetime(2024, 1, 1)
end_date   = datetime.datetime(2025, 12, 31)

df_LST = generate_LST_dataframe_by_year(data_dir, shapefile_path, start_date, end_date)
df_LST.to_csv("LST_2001_2025_provincia_pivot.csv", index=False)
print(df_LST.head())

# 3. MONTHLY PM2.5 

In [ ]:
def generate_PM25_monthly(data_dir, shapefile_path, start_date, end_date):
    """
    Genera un DataFrame pivot con promedio mensual de PM2.5 por departamento.
    """

    # 1️⃣ Leer shapefile y asegurar CRS WGS84
    gdf = gpd.read_file(shapefile_path).to_crs("EPSG:4326")

    # 2️⃣ Listar archivos .nc
    nc_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".nc")])

    df_final = pd.DataFrame()

    # 3️⃣ Iterar sobre cada archivo
    for file in nc_files:
        base = os.path.basename(file)
        yyyymm = base.split('.')[-2].split('-')[0]
        year = int(yyyymm[:4])
        month = int(yyyymm[4:6])
        file_date = datetime.datetime(year, month, 1)

        # Filtrar por rango de fechas
        if not (start_date <= file_date <= end_date):
            continue

        print(f"Procesando: {file_date.strftime('%Y-%m')}")

        # Abrir dataset
        ds = xr.open_dataset(file)
        da = ds['PM25']

        # Convertir a rioxarray
        da = da.rio.write_crs("EPSG:4326")
        da.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)

        results_list = []

        # Iterar por cada departamento
        for idx, row in gdf.iterrows():
            dep = row["NOMBDEP"]
            geom = [mapping(row["geometry"])]
            try:
                clipped = da.rio.clip(geom, gdf.crs, drop=True, all_touched=True)
                mean_val = float(clipped.mean().values)
            except Exception:
                mean_val = pd.NA
            results_list.append({"NOMBDEP": dep, "date": file_date, "PM25_mean": mean_val})

        ds.close()
        del da

        # Crear DataFrame y pivot
        # Crear DataFrame
        df_date = pd.DataFrame(results_list)
        df_date["col_name"] = df_date["date"].dt.strftime("PM25_mean_%Y_%m")

        # Usar pivot_table para ignorar duplicados y calcular promedio si hay más de un valor
        df_pivot = df_date.pivot_table(
            index="NOMBDEP",
            columns="col_name",
            values="PM25_mean",
            aggfunc='mean',   # o 'first', 'max', según lo que prefieras
            fill_value=pd.NA
        ).reset_index()


        # Merge con df_final
        if df_final.empty:
            df_final = df_pivot
        else:
            df_final = pd.merge(df_final, df_pivot, on="NOMBDEP", how="outer")

    return df_final

In [ ]:
# -------------------------------
data_dir = r"D:\Documents_2\LABORAL\2025\INNOVALAB\data\pm25\Peru_only"
shapefile_path = r"D:\Documents_2\LABORAL\2025\INNOVALAB\shapefile\DEPARTAMENTOS_inei_geogpsperu_suyopomalia.shp"

start_date = datetime.datetime(1998, 1, 1)
end_date   = datetime.datetime(2023, 12, 31)

df_PM25 = generate_PM25_monthly(data_dir, shapefile_path, start_date, end_date)

# Guardar CSV
df_PM25.to_csv("PM25_provincias_pivot.csv", index=False)
print(df_PM25.head())

# 4. NO2 

In [4]:
def generate_NO2_dataframe(data_dir, shapefile_path, start_date, end_date):
    """
    Genera un DataFrame pivot con promedio mensual de NO2 por departamento
    usando archivos .tif con nombres tipo NO2_Peru_YYYY_MM.tif
    """

    # Leer shapefile y asegurar CRS WGS84
    gdf = gpd.read_file(shapefile_path).to_crs("EPSG:4326")

    # Listar archivos .tif
    tif_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".tif")])

    df_final = pd.DataFrame()

    for file in tif_files:
        base = os.path.basename(file)
        
        # Extraer año y mes del nombre: NO2_Peru_YYYY_MM.tif
        parts = base.split('_')
        year = int(parts[2])
        month = int(parts[3].split('.')[0])
        file_date = datetime.datetime(year, month, 1)

        # Filtrar por rango de fechas
        if not (start_date <= file_date <= end_date):
            continue

        print(f"Procesando: {file_date.strftime('%Y-%m')}")

        # Abrir raster con rioxarray
        da = rioxarray.open_rasterio(file)
        da = da.rio.write_crs("EPSG:4326")

        # Tomar primera banda si existe
        if "band" in da.dims:
            da = da.isel(band=0)

        results_list = []

        # Iterar por cada departamento
        for idx, row in gdf.iterrows():
            dep = row["NOMBDEP"]
            geom = [mapping(row["geometry"])]
            try:
                clipped = da.rio.clip(geom, gdf.crs, drop=True, all_touched=True)
                mean_val = float(clipped.mean().values)
            except Exception:
                mean_val = pd.NA
            results_list.append({"NOMBDEP": dep, "date": file_date, "NO2_mean": mean_val})

        # Crear DataFrame y pivot usando pivot_table para ignorar duplicados
        df_date = pd.DataFrame(results_list)
        df_date["col_name"] = df_date["date"].dt.strftime("NO2_mean_%Y_%m")
        df_pivot = df_date.pivot_table(
            index="NOMBDEP",
            columns="col_name",
            values="NO2_mean",
            aggfunc='mean',  # si hay duplicados, tomar promedio
            fill_value=pd.NA
        ).reset_index()

        # Asegurar que todos los departamentos del shapefile estén presentes
        df_pivot = gdf[["NOMBDEP"]].merge(df_pivot, on="NOMBDEP", how="left")

        # Merge con df_final
        if df_final.empty:
            df_final = df_pivot
        else:
            df_final = pd.merge(df_final, df_pivot, on="NOMBDEP", how="outer")

    return df_final


In [5]:
# -------------------------------
area= 'DEPARTAMENTOS' # # CAMBIAR POR PROVINCIAS, DISTRITOS Y DEPARTAMENTOS
data_dir = r"D:\Documents_2\LABORAL\2025\INNOVALAB\data\no2"
shapefile_path = rf"D:\Documents_2\LABORAL\2025\INNOVALAB\shapefile\{area}_inei_geogpsperu_suyopomalia.shp" 

start_date = datetime.datetime(2024, 6, 1)
end_date   = datetime.datetime(2024, 12, 31)

df_NO2 = generate_NO2_dataframe(data_dir, shapefile_path, start_date, end_date)

# Guardar CSV
df_NO2.to_csv("NO2_2018_2024_provincias_pivot.csv", index=False)
print(df_NO2.head())

Procesando: 2024-06
Procesando: 2024-07
Procesando: 2024-08
Procesando: 2024-09
Procesando: 2024-10
Procesando: 2024-11
Procesando: 2024-12
    NOMBDEP  NO2_mean_2024_06  NO2_mean_2024_07  NO2_mean_2024_08  \
0  AMAZONAS          0.000004          0.000005          0.000007   
1    ANCASH          0.000011          0.000012          0.000014   
2  APURIMAC          0.000005          0.000007          0.000008   
3  AREQUIPA          0.000010          0.000012          0.000012   
4  AYACUCHO          0.000004          0.000006          0.000008   

   NO2_mean_2024_09  NO2_mean_2024_10  NO2_mean_2024_11  NO2_mean_2024_12  
0          0.000008          0.000007          0.000008          0.000004  
1          0.000013          0.000010          0.000011          0.000009  
2          0.000007          0.000008          0.000009          0.000006  
3          0.000011          0.000011          0.000007          0.000006  
4          0.000007          0.000006          0.000006          